### Using Draft Year and Player Names for Data Integration
In the process of enriching our NBA dataset with player position information, we faced a challenge in accurately merging data from the NBA API with our existing dataset. Our existing dataset lacked a unique player identifier, such as a player ID, which is typically used to ensure precise matching in data integration tasks.

#### Why Draft Year and Player Names?
To overcome this challenge, we chose to use a combination of player names and draft years as key identifiers for merging the datasets. This decision was based on the following considerations:  

Uniqueness: Although player names alone can sometimes lead to duplication (due to common names shared by multiple players), combining names with draft years significantly increases the likelihood of creating a unique identifier for each player. It's rare for two players with the exact same name to be drafted in the same year.  

Data Availability: Both our existing dataset and the data fetched from the NBA API consistently include player names and draft years. This consistency is crucial for creating reliable merge keys.  

Simplicity and Practicality: Using available data fields without the need for complex transformations or external data sources simplifies the data integration process. It also aligns with practical constraints and available resources.  

#### Implementation Approach
We implemented this approach by first creating a concatenated key in both datasets, combining each player's full name with their draft year (formatted as a string). For example, a player named "John Doe" drafted in 1998 would have a merge key of "John Doe_1998". This key was then used to merge the datasets, ensuring that each player's position information from the NBA API correctly aligned with their respective entry in our existing dataset.

In [1]:
from nba_api.stats.endpoints import PlayerIndex
import pandas as pd
import numpy as np

# Initialize the PlayerIndex class to fetch data
player_index = PlayerIndex(historical_nullable='1')

# Extract the player index data
player_data = player_index.player_index.get_data_frame()

# Select only the necessary columns and explicitly create a copy
selected_columns = ['DRAFT_YEAR', 'PLAYER_LAST_NAME', 'PLAYER_FIRST_NAME', 'POSITION']
player_position_data = player_data[selected_columns].copy()

# Fill missing values in 'DRAFT_YEAR', convert to integer, then to string
player_position_data['DRAFT_YEAR'] = player_position_data['DRAFT_YEAR'].fillna(0).astype(int).astype(str)

# Concatenate the first and last names to create a full name
player_position_data['full_name'] = player_position_data['PLAYER_FIRST_NAME'] + ' ' + player_position_data['PLAYER_LAST_NAME']

# Create a combined key using full name and draft year
player_position_data['merge_key'] = player_position_data['full_name'] + '_' + player_position_data['DRAFT_YEAR']

# Save the fetched data
player_position_data.to_csv("./OriginalDatasets/player_positions.csv", index=False)

In [2]:
player_position_data.head()

,DRAFT_YEAR,PLAYER_LAST_NAME,PLAYER_FIRST_NAME,POSITION,full_name,merge_key
0,1990,Abdelnaby,Alaa,F,Alaa Abdelnaby,Alaa Abdelnaby_1990
1,1968,Abdul-Aziz,Zaid,C,Zaid Abdul-Aziz,Zaid Abdul-Aziz_1968
2,1969,Abdul-Jabbar,Kareem,C,Kareem Abdul-Jabbar,Kareem Abdul-Jabbar_1969
3,1990,Abdul-Rauf,Mahmoud,G,Mahmoud Abdul-Rauf,Mahmoud Abdul-Rauf_1990
4,1997,Abdul-Wahad,Tariq,F-G,Tariq Abdul-Wahad,Tariq Abdul-Wahad_1997


In [3]:
# Load the existing dataset
existing_data = pd.read_csv('./ExsitingDatasets/all_seasons.csv')

# Create a combined key in the existing data using player_name and draft_year
existing_data['merge_key'] = existing_data['player_name'] + '_' + existing_data['draft_year']

In [4]:
# Merge the datasets on the merge_key
merged_data = pd.merge(existing_data, player_position_data[['merge_key', 'POSITION']], 
                       on='merge_key', how='left')

# Drop the extra 'merge_key' column after merge, if no longer needed
merged_data.drop('merge_key', axis=1, inplace=True)

In [5]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12845 entries, 0 to 12844
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         12845 non-null  int64  
 1   player_name        12845 non-null  object 
 2   team_abbreviation  12845 non-null  object 
 3   age                12845 non-null  float64
 4   player_height      12845 non-null  float64
 5   player_weight      12845 non-null  float64
 6   college            10991 non-null  object 
 7   country            12845 non-null  object 
 8   draft_year         12845 non-null  object 
 9   draft_round        12845 non-null  object 
 10  draft_number       12845 non-null  object 
 11  gp                 12845 non-null  int64  
 12  pts                12845 non-null  float64
 13  reb                12845 non-null  float64
 14  ast                12845 non-null  float64
 15  net_rating         12845 non-null  float64
 16  oreb_pct           128

#### Addressing Missing POSITION Data in Merged NBA Dataset
Upon merging our existing NBA dataset with player position data from the NBA API using player names and draft years as key identifiers, we observed that some players in the merged dataset were missing POSITION information. This discrepancy primarily arises from missing or mismatched draft_year data.   

Missing draft_year in API Data: Some players in the NBA API's PlayerIndex data might not have draft_year information. This absence occurs particularly with undrafted players or those with incomplete records in the API.  

We can add undrafted players into consideration by doing the following.

In [6]:
# Load the existing dataset
existing_data = pd.read_csv('./ExsitingDatasets/all_seasons.csv')

# Replace 'Undrafted' with '0' in the 'draft_year' column of your existing dataset
existing_data['draft_year'] = existing_data['draft_year'].replace('Undrafted', '0')

# Create a combined key in the existing data using player_name and draft_year
existing_data['merge_key'] = existing_data['player_name'] + '_' + existing_data['draft_year']

# Assuming 'player_position_data' is already prepared with the 'merge_key'
# Merge the datasets on the merge_key
merged_data = pd.merge(existing_data, player_position_data[['merge_key', 'POSITION']], 
                       on='merge_key', how='left')

# Drop the extra 'merge_key' column after merge, if no longer needed
merged_data.drop('merge_key', axis=1, inplace=True)

# Remove duplicates
merged_data = merged_data.drop_duplicates(subset=['Unnamed: 0'])

In [7]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12844 entries, 0 to 12872
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         12844 non-null  int64  
 1   player_name        12844 non-null  object 
 2   team_abbreviation  12844 non-null  object 
 3   age                12844 non-null  float64
 4   player_height      12844 non-null  float64
 5   player_weight      12844 non-null  float64
 6   college            10990 non-null  object 
 7   country            12844 non-null  object 
 8   draft_year         12844 non-null  object 
 9   draft_round        12844 non-null  object 
 10  draft_number       12844 non-null  object 
 11  gp                 12844 non-null  int64  
 12  pts                12844 non-null  float64
 13  reb                12844 non-null  float64
 14  ast                12844 non-null  float64
 15  net_rating         12844 non-null  float64
 16  oreb_pct           12844 no

#### Simplification of Player Positions in the NBA Dataset
We encountered a situation where some players were listed with multiple positions (e.g., "G-F" for Guard-Forward or "C-F" for Center-Forward). This multiposition notation reflects the versatile roles that players can fulfill on the basketball court. However, for the purposes of streamlined analysis and consistency within our dataset, we decided to simplify the position notation by retaining only the primary position for each player.

In [8]:
cleaned_data = merged_data.dropna(subset=['POSITION'])

# Now 'cleaned_data' contains only rows where 'POSITION' is not null

In [9]:
cleaned_data = cleaned_data.copy()

cleaned_data['POSITION'] = cleaned_data['POSITION'].apply(lambda x: x.split('-')[0])

In [10]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12687 entries, 0 to 12872
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         12687 non-null  int64  
 1   player_name        12687 non-null  object 
 2   team_abbreviation  12687 non-null  object 
 3   age                12687 non-null  float64
 4   player_height      12687 non-null  float64
 5   player_weight      12687 non-null  float64
 6   college            10869 non-null  object 
 7   country            12687 non-null  object 
 8   draft_year         12687 non-null  object 
 9   draft_round        12687 non-null  object 
 10  draft_number       12687 non-null  object 
 11  gp                 12687 non-null  int64  
 12  pts                12687 non-null  float64
 13  reb                12687 non-null  float64
 14  ast                12687 non-null  float64
 15  net_rating         12687 non-null  float64
 16  oreb_pct           12687 no

In [11]:
categoricals = cleaned_data.select_dtypes(exclude=[np.number])
categoricals.describe()

,player_name,team_abbreviation,college,country,draft_year,draft_round,draft_number,season,POSITION
count,12687,12687,10869,12687,12687,12687,12687,12687,12687
unique,2488,36,349,80,48,9,75,27,3
top,Vince Carter,CLE,Kentucky,USA,0,1,Undrafted,2021-22,G
freq,22,446,439,10600,2277,7297,2332,597,5396


In [12]:
# Save the merged dataset to a new CSV file
cleaned_data.to_csv('./OriginalDatasets/all_seasons_position.csv', index=False)